### RSM338 Assignment 1


In [ ]:
# Import modules
!pip install pandas
import pandas as pd
import numpy as np

#### Question 1

In [136]:
# import monthly data
monthly = pd.read_csv('./monthly.csv',header=None)
date = monthly.iloc[:,0]
monthly = monthly.drop(0,axis=1)
monthly.columns = ['VW','EW']+['DEC{}'.format(i) for i in range(1, 11)]  # Specify the names of the columns

In [ ]:
monthly.head()

### Here is the code for generating some test statistics for the portfolios

In [ ]:
monthly.describe()

#### Question 1b.

Computing the skew and (excess) kurtosis on the monthly returns of the 12 portfolios. Including a test of normality for each coefficient.

In [ ]:
# Importing scipy stats library for skewness, kurtosis, and normality test
!pip install scipy
import scipy.stats as stats

In [ ]:
# Find the coefficient of skewness for the monthly returns across each column
# Normality Test for coefficient by finding z-score and normalizing
skewness = monthly.apply(stats.skew)
skewness = skewness.to_frame()
skewness.rename(columns={0: 'Skewness'}, inplace=True)
skewness['Z1'] = skewness['Skewness']/np.sqrt(6/1176)
skewness['P-value'] = 1 - skewness['Z1'].apply(stats.norm.cdf)

skewness

Both skew and kurtosis have integrated tests for normality within the `scipy` library called `skewtest` and `kurtosistest` respectively.

These calculate the z-score and the P-value for each coefficient however I decided to compute them manually and form them into a table.

In [ ]:
# Coefficient of excess kurtosis and a Normality Test
kurtosis = monthly.apply(stats.kurtosis)
coefficients = kurtosis.to_frame()
coefficients.rename(columns={0: 'Kurtosis'}, inplace=True)
coefficients['Z2'] = coefficients['Kurtosis']/np.sqrt(24/1176)
coefficients['P-value'] = 1 - coefficients['Z2'].apply(stats.norm.cdf)

coefficients

#### Question 1c.

Repeat the steps in parts a) and b), but for just January and the rest of the months separately.

Group the data by Jan and not Jan.

In [ ]:
# Reimport monthly data and separate date index
monthly = pd.read_csv('./monthly.csv',header=None)
date = monthly.iloc[:,0]
monthly = monthly.drop(0,axis=1) # remove date column
monthly.columns = ['VW','EW']+['DEC{}'.format(i) for i in range(1, 11)]  # Specify the names of the columns

monthly.head()

In [ ]:
date = pd.to_datetime(date, format='%Y%m%d')

monthly.head()

In [ ]:
# Turn date Series into DateTime index for monthly df
date = pd.to_datetime(date, format='%Y%m%d')
monthly.index = date

monthly.head()

In [ ]:
# Create columns that identify year, month, day for each row
monthly['month'] = monthly.index.month
monthly = monthly.reset_index(drop=True)

monthly.head()

In [146]:
# Separate the January Data from the rest of the data
columns = ['VW','EW']+['DEC{}'.format(i) for i in range(1, 11)]

jan_data = monthly[monthly['month'] == 1].drop(columns='month')
non_jan_data = monthly[monthly['month'] != 1].drop(columns='month')


In [ ]:
# Compute test statistics for both Jan and non Jan data
print("January Test Statistics")
print(jan_data.describe())

# Non Jan data
print("Non January Test Statistics")
print(non_jan_data.describe()) 

Repeating part b) of the question incorporating `scipy.stats` to find skewness and kurtosis for each of the 12 months


In [ ]:
import scipy.stats as stats
# Finding the coefficients and the normality tests as done above
jan_skewness = jan_data.apply(stats.skew)
print("January Skewness Coefficients")
print(jan_skewness)

# Conduct Normality Test
jan_normality_skew = jan_data.apply(stats.skewtest)
print('January Skewness Tests')
for column, row in jan_normality_skew.items():
    test, pvalue = row
    print(f"{column}: Z-score = {test: 4f}, P-value = {pvalue: 4f}")


In [ ]:
# Compute the same for non Jan skewness
non_jan_skewness = non_jan_data.apply(stats.skew)
print("Non January Skewness Coefficients")
print(non_jan_skewness)

# Conduct Normality Test
non_jan_normality_skew = non_jan_data.apply(stats.skewtest)
print('Non January Skewness Tests')
for column, row in non_jan_normality_skew.items():
    test, pvalue = row
    print(f"{column}: Z-score = {test: 4f}, P-value = {pvalue: 4f}")

##### Compute the kurtosis for both slices of the data

In [ ]:
jan_kurtosis = jan_data.apply(stats.kurtosis)
print('Jan Kurtosis Coefficients and Normality Tests')

# Normality tests for Kurtosis
jan_coefficients = jan_kurtosis.to_frame()
jan_coefficients.rename(columns={0: 'Excess Kurtosis'}, inplace=True)
jan_coefficients['Z2'] = jan_coefficients['Excess Kurtosis']/np.sqrt(24/98)
jan_coefficients['P-value'] = 1 - jan_coefficients['Z2'].apply(stats.norm.cdf)

print(jan_coefficients)


In [ ]:
# Coefficients for Non Jan data
non_jan_kurtosis = non_jan_data.apply(stats.kurtosis)
print("Non Jan Kurtosis Coefficients and Normality Tests")

# Normality Tests
non_jan_coefficients = non_jan_kurtosis.to_frame()
non_jan_coefficients.rename(columns={0: 'Excess Kurtosis'}, inplace=True)
non_jan_coefficients['Z2'] = non_jan_coefficients['Excess Kurtosis']/np.sqrt(24/1078)
non_jan_coefficients['P-value'] = 1 - non_jan_coefficients['Z2'].apply(stats.norm.cdf)

print(non_jan_coefficients)


#### Question 2

In [152]:
import datetime

In [153]:
# Input the data
daily = pd.read_csv('./daily.csv',header=None)

# change column names
daily.columns = ['date', 'value-weighted', 'equal-weighted']

# convert date into datetime object
daily['date'] = pd.to_datetime(daily['date'], format='%Y%m%d') # http://strftime.org/

### create variable weekday that represents the day in the date column
### (NOTE: python count starts at 0)
weekday = daily['date'].dt.dayofweek
week_label = ['Monday','Tuesday','Wednesday','Thursday','Friday', 'Saturday']

In [ ]:
# Mapping the week_labels to weekday Series
weekday = weekday.map(lambda x: week_label[x])
weekday.head(10)


In [ ]:
# Create a column for the weekday names for easier indexing
daily['weekday'] = weekday

#Filtering for wednesdays and finding the missing wednesdays
wednesdays = daily[daily['weekday'] == 'Wednesday']
week_after_missing = wednesdays.loc[wednesdays['date'] - wednesdays['date'].shift(1) > datetime.timedelta(days=7)]
missing_week = week_after_missing['date'] - datetime.timedelta(days=7)

# Get potential next dates (Thursday or Tuesday)
next_thurs = missing_week + datetime.timedelta(days=1)
next_tues = missing_week - datetime.timedelta(days=1)

missing_week

##### Use the data with the missing weeks to find which Tuesdays and Thursdays, from the weeks that have no data for Wednesday, exist in the daily.csv file.

##### Create a series so that we can use this as a boolean mask to index daily

In [ ]:
# Finding the Thursdays that exist in the daily.csv 
thursdays = daily.loc[daily['date'].isin(missing_week + datetime.timedelta(days=1))]

# Using the exisiting Thursdays to find which weeks we still need to find
missing_thursdays = next_thurs.loc[~(next_thurs.isin(thursdays['date']))]
# Find the Tuesdays that we need based of the missing Thursdays
needed_tuesdays = missing_thursdays - datetime.timedelta(days=2)

# Create a slice of the dataframe that includes each Wednesday, including the Thursdays or Tuesdays for missing weeks
weekly = daily[(daily['date'].isin(wednesdays['date']))|(daily['date'].isin(thursdays['date']))|(daily['date'].isin(needed_tuesdays))]

weekly['vw_return'] = ((1 + weekly['value-weighted'])/(1+ weekly['value-weighted'].shift(1)) - 1)  # Creating the value-weighted weekly returns
weekly['ew_return'] = ((1 + weekly['equal-weighted'])/(1 + weekly['equal-weighted'].shift(1)) - 1)  # Creating the equal-weighted weekly returns
weekly = weekly.drop(columns=['value-weighted', 'equal-weighted'])
weekly.head()


### Question 2 Part B - Computing test statistics

In [ ]:
weekly.describe().drop(columns=['date'])

### Question 2 Part C - Skew, Kurtosis and Normality Tests

In [ ]:
portfolios = weekly.drop(columns=['date', 'weekday'])
# Dropping the 'date' and 'weekday' column and first NaN value so that we can run stats.skew
weekly_nonan = portfolios.dropna()

# Creating Coefficients
coefficients = weekly_nonan.apply(stats.skew)

# Create the DataFrame
skew_coef = coefficients.to_frame()
skew_coef.rename(columns={0: 'Skewness'}, inplace=True)


In [ ]:
# Skew Coefficients
print('Weekly Returns Skewness Coefficients')
print(skew_coef)

# Conducting Normality Tests
skew_normality = weekly_nonan.apply(stats.skewtest)
print("Weekly Returns Skewness Normality Tests")
for column, row in skew_normality.items():
    test, pvalue = row
    print(f"{column}: Z-score = {test: 4f}, P-value = {pvalue: 4f}")

In [ ]:
# Create kurtosis coefficients
coefficients = weekly_nonan.apply(stats.kurtosis)
kurt_coeff = coefficients.to_frame()
kurt_coeff.rename(columns={0: 'Excess Kurtosis'}, inplace = True)

# Calculate the z2 and P-value columns for Normality test
kurt_coeff['z2'] = kurt_coeff['Excess Kurtosis']/np.sqrt(24/5077)
kurt_coeff['P-value'] = 1 - kurt_coeff['z2'].apply(stats.norm.cdf) # One tailed test to find P-value

print('Weekly Return Kurtosis Coefficients and Normality Test')
print(kurt_coeff)